In [1]:
from imctools.scripts import ometiff2analysis
from imctools.scripts import cropobjects
from imctools.scripts import croprandomsection
from imctools.scripts import resizeimage
from imctools.scripts import generatedistancetospheres
from imctools.scripts import imc2tiff
from imctools.scripts import ome2micat
from imctools.scripts import probablity2uncertainty
from imctools.scripts import convertfolder2imcfolder

In [2]:
import os
import re
import zipfile

In [3]:
%load_ext rpy2.ipython


# The IMC preprocessing pipeline for multiplexed image analysis


In [4]:
# the folders with the txt/mcd files for the analysis

# this can be
#folders_multiac = '/home/imcuser/Data/20170905_example_dataset/Acquisitions/'
#folders = [os.path.join(folders_multiac, fol) for fol in os.listdir(folders_multiac)]
folders = ['/home/imcuser/Data/20170905_example_dataset/Acquisitions/20170906_FluidigmONfinal_SE/']

# part that all considered files need to have in common
common_filepart = '.txt'

# output for OME tiffs
folder_base = '/home/imcuser/Data/20170905_example_dataset/Analysis'
folder_analysis = os.path.join(folder_base, 'tiffs')
folder_ilastik = os.path.join(folder_base, 'ilastik')
folder_imczip = os.path.join(folder_base, 'imczip')
folder_ome = os.path.join(folder_base, 'ometiff')
folder_cp = os.path.join(folder_base, 'cpout')
folder_histocat = os.path.join(folder_base, 'histocat')
folder_uncertainty = os.path.join(folder_base, 'uncertainty')


# pannel
csv_pannel = '/home/imcuser/Data/20170905_example_dataset/Metadata/20170906_IMCexample_pannel.csv'
csv_pannel_metal = 'Metal Tag'
csv_pannel_ilastik = 'ilastik'
# Explicitly indicates which metals should be used for the full stack
csv_pannel_full = 'full'

# spillover matrix
csv_spillmat = '/home/imcuser/Data/20170905_example_dataset/Metadata/20170707_example_spillmat.csv'
csv_spillmatout = os.path.join(folder_analysis,'sm_full')

# parameters for resizing the images for ilastik
suffix_full = '_full'
suffix_ilastik = '_ilastik'
suffix_ilastik_scale = '_s2'
suffix_mask = '_mask.tiff'
suffix_probablities = '_probabilities'


failed_images = list()

Specify which steps to run

In [5]:
do_convert_imc = True
do_convert_txt = True
do_stacks = True
do_ilastik = True
do_histocat = True

Generate all the folders if necessary

In [6]:
for fol in [folder_base, folder_analysis, folder_ilastik,
            folder_imczip, folder_ome, folder_cp, folder_histocat, folder_uncertainty]:
    if not os.path.exists(fol):
        os.makedirs(fol)

Optional Beta feature: Convert folders to _imc.zip folder, containing all the metadata contained in an .mcd

In [7]:
if do_convert_imc:
    for fol in folders:
        try:
            convertfolder2imcfolder.convert_folder2imcfolder(fol, folder_imczip)
        except:
            print('Failed Folder: ' + fol)

Convert extract the .txt files to .ome.tiff

In [8]:
if do_convert_txt:
    for fol in folders:
        for fn in os.listdir(fol):
            # check if the files already exist, only convert if they do not.
            if len([f for f in os.listdir(folder_ome) if (fn.rstrip('.txt').rstrip('.mcd') in f)]) == 0:
                if common_filepart in fn: # and 'tuningtape' not in fn:
                    txtname = os.path.join(fol, fn)
                    try:
                        imc2tiff.save_imc_to_tiff(txtname,tifftype='ome', outpath=folder_ome)
                        print('Converted : '+fn)
                    except:
                        failed_images.append(txtname)
                        print('Conversion failed: ' + txtname)
            

Convert ome.tiffs to a HistoCAT compatible format, e.g. to do some visualization and channel checking.

In [9]:
if do_histocat:
    if not(os.path.exists(folder_histocat)):
        os.makedirs(folder_histocat)
    ome2micat.omefolder2micatfolder(folder_ome, folder_histocat, dtype='uint16')
# The bigtiff warning can be ignored

/usr/local/lib/python2.7/dist-packages/tifffile/tifffile.py:452: UserWarning: writing incompatible bigtiff ImageJ
  warnings.warn("writing incompatible bigtiff ImageJ")


Generate the analysis stacks

In [10]:
if do_stacks:
    for img in os.listdir(folder_ome):
        if not img.endswith('.ome.tiff'):
            pass
        basename = img.rstrip('.ome.tiff')
        print(img)
        ometiff2analysis.ometiff_2_analysis(os.path.join(folder_ome, img), folder_analysis, basename+suffix_full,
                                           pannelcsv=csv_pannel, metalcolumn=csv_pannel_metal,
                                            usedcolumn=csv_pannel_full, bigtiff=False)



Site2_left_lower_2_3_a0.ome.tiff
Site4_lower_right_1_4_a0.ome.tiff
Site3_upper_right_1_6_a0.ome.tiff
Site4_lower_right_2_5_a0.ome.tiff
Site3_upper_right_2_7_a0.ome.tiff
Site1_left_upper_2_1_a0.ome.tiff
Site1_left_upper_1_0_a0.ome.tiff
Site2_left_lower_1_2_a0.ome.tiff


Generate the ilastik stacks

In [11]:
if do_ilastik:
    for img in os.listdir(folder_ome):
        if not img.endswith('.ome.tiff'):
            pass
        basename = img.rstrip('.ome.tiff')
        ometiff2analysis.ometiff_2_analysis(os.path.join(folder_ome, img), folder_analysis,
                                            basename + suffix_ilastik, pannelcsv=csv_pannel, metalcolumn=csv_pannel_metal,
                                            usedcolumn=csv_pannel_ilastik, addsum=True, bigtiff=False)

-> Before the next step run the cellprofiler 'ilastik preprocessing' pipeline to generate a stacks for ilastik that are 2x enalarged and have hot pixels removed

Then train the Ilastik classifier in 3 classes for pixel segmentation.
- Nuclei
- Cytoplasm/Membrane
- Background

Usually it is best to label very sparsely to avoid creating a to large but redundant training data set. After initially painting few pixels, check the uncertainty frequently and only paint pixels with high uncertainty.

Once this looks nice for all the cropped sections, batch process the whole images using the code bellow. 

## Run the ilastik classification as a batch

In [12]:
fn_ilastikproject = '/home/imcuser/Data/20170905_example_dataset/Analysis/ilastik/cellular_pixel_classification_v1.ilp'
bin_ilastik = "/home/imcuser/Bin/ilastik-1.2.2-Linux/run_ilastik.sh"
n_mb_ram = 7000
n_cores = 2

In [13]:
fn_ilastik_input =os.path.join(folder_analysis,"*"+suffix_ilastik_scale+'.tiff')
glob_probabilities = os.path.join(folder_analysis,"{nickname}"+suffix_probablities+'.tiff')

In [14]:
%%bash -s "$bin_ilastik" "$fn_ilastikproject" "$glob_probabilities" "$fn_ilastik_input" "$n_mb_ram" "$n_cores"
echo $1
echo $2
echo $3
echo $4
echo $5
echo $6
date +"%T"
LAZYFLOW_TOTAL_RAM_MB=$5 \
LAZYFLOW_THREADS=$6\
    $1 \
    --headless --project=$2 \
    --output_format=tiff \
    --output_filename_format=$3 \
    --export_dtype uint16 --pipeline_result_drange="(0.0, 1.0)" \
    --export_drange="(0,65535)" $4
date +"%T"

/home/imcuser/Bin/ilastik-1.2.2-Linux/run_ilastik.sh
/home/imcuser/Data/20170905_example_dataset/Analysis/ilastik/cellular_pixel_classification_v1.ilp
/home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/{nickname}_probabilities.tiff
/home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site1_left_upper_1_0_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site1_left_upper_2_1_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site2_left_lower_1_2_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site2_left_lower_2_3_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site3_upper_right_1_6_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site3_upper_right_2_7_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site4_lower_right_1_4_a0_ilastik_s2.tiff /home/imcuser/Data/20170905_example_dataset/Analysis/tiffs/Site4

WARNING cross_validation.py(44): DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
WARNING 2017-09-14 20:46:49,179 opConservationTracking 11620 139951817070400 Could not find any ILP solver
WARNING 2017-09-14 20:46:49,321 opStructuredTracking 11620 139951817070400 Could not find any ILP solver
WARNING 2017-09-14 20:46:49,334 structuredTrackingWorkflow 11620 139951817070400 Could not find any learning solver (HYTRA, PGMLINK). Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!
WARNING pkgutil.py(186): ImportWarning: Not importing directory '/home/imcuser/Bin/ilastik-1.2.2-Linux/lib/python2.7/site-packages/mpl_toolkits': missing __init__.py


## Generate the spillovermatrix for cellprofiler compensation

In [15]:
%%R -i csv_pannel -i csv_pannel_metal -i csv_pannel_full -i csv_spillmat -i csv_spillmatout
metal_col = make.names(csv_pannel_metal)
full_col = make.names(csv_pannel_full)

pannel_dat = read.csv(csv_pannel)
analysis_channels = pannel_dat[pannel_dat[,full_col] ==T,metal_col]

analysis_channels = paste(analysis_channels, 'Di', sep = '')

sm = as.matrix(read.csv(csv_spillmat, row.names=1))

sm_table = CATALYST::adaptSpillmat(sm, analysis_channels)

write.table(sm_table, paste0(csv_spillmatout,'.csv'))
tiff::writeTIFF(sm_table, paste0(csv_spillmatout,'.tiff'), bits.per.sample = 32, reduce = T)

/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: WARNING: Compensation is likely to be inaccurate.
         Spill values for the following interactions
         have not been estimated:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Ru100Di -> Ru102Di

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Ru102Di -> Ru100Di

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Yb176Di -> Er168Di, Er170Di, Lu175Di, Yb171Di, Yb172Di, Yb174Di

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Ir191Di -> Ir193Di

  warnings.warn(x, RRuntimeWarning)


[1] 1


## convert probabilities to uncertainties

In [16]:
for fn in os.listdir(folder_analysis):
    if fn.endswith(suffix_probablities+'.tiff'):
        probablity2uncertainty.probability2uncertainty(os.path.join(folder_analysis,fn), folder_uncertainty)

## Generate the micat folder

In [17]:
if do_histocat:
    if not(os.path.exists(folder_histocat)):
        os.makedirs(folder_histocat)
    ome2micat.omefolder2micatfolder(folder_ome, folder_histocat, fol_masks=folder_cp, mask_suffix=suffix_mask, dtype='uint16')